# Book Apps

#### Descripción

El coronavirus tomó al mundo entero por sorpresa, cambiando la rutina diaria de todos y todas. Los habitantes de las ciudades ya no pasaban su tiempo libre fuera, yendo a cafés y centros comerciales; sino que más gente se quedaba en casa, leyendo libros. Eso atrajo la atención de las startups (empresas emergentes) que se apresuraron a desarrollar nuevas aplicaciones para los amantes de los libros.

En este proyecto se trabajará con una base de datos de uno de los servicios que compiten en este mercado. Contiene datos sobre libros, editoriales, autores y calificaciones de clientes y reseñas de libros. Esta información se utilizará para generar una propuesta de valor para un nuevo producto.

#### Diccionario de datos:

**`books`**

Contiene datos sobre libros:

- `book_id`: identificación del libro
- `author_id`: identificación del autor o autora
- `title`: título
- `num_pages`: número de páginas
- `publication_date`: fecha de la publicación
- `publisher_id`: identificación de la editorial

**`authors`**

Contiene datos sobre autores:

- `author_id`: identificación del autor o autora
- `author`: el autor o la autora

**`publishers`**

Contiene datos sobre editoriales:

- `publisher_id`: identificación de la editorial
- `publisher`: la editorial

**`ratings`**

Contiene datos sobre las calificaciones de usuarios:

- `rating_id`: identificación de la calificación
- `book_id`: identificación del libro
- `username`: el nombre del usuario que revisó el libro
- `rating`: calificación

**`reviews`**

Contiene datos sobre las reseñas de los y las clientes:

- `review_id`: identificación de la reseña
- `book_id`: identificación del libro
- `username`: el nombre del usuario que revisó el libro
- `text`: el texto de la reseña

In [2]:
# !/usr/bin/python

%pip install python-dotenv

# importar librerías
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

load_dotenv()

db_config = {
    'user': os.getenv('DB_USER'),
    'pwd': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT'),
    'db': os.getenv('DB_NAME')
}


connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                        db_config['pwd'],
                                                        db_config['host'],
                                                        db_config['port'],
                                                        db_config['db'])


# código de muestra para conectarse a la base de datos
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

#Creo una función donde selecciono todas las columnas, y según el nombre de la tabla que selecciono obtengo los datos de la misma

def cargar_tabla(name, mostrar=True):
    query = f"SELECT * FROM {name}"
    df = pd.read_sql(query, con=engine)
    
    if mostrar:
        display(df.head(5))
    
    return df


# llamo a los dfs
books       = cargar_tabla("books")
authors     = cargar_tabla("authors")
publishers  = cargar_tabla("publishers")
ratings     = cargar_tabla("ratings")
reviews     = cargar_tabla("reviews")

Note: you may need to restart the kernel to use updated packages.


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Se recibieron 5 tablas, donde la mayor cantidad de datos se guardan en la llamada 'books'. Para poder hacer ciertos análisis vamos a trabajar combinándolas, quizás utilizando la tabla mencionada como puente entre ellas.

In [ ]:
#Cantidad de libros publicados después del 2000

query_2000 = ''' 
            SELECT 
                title,
                publication_date
            FROM 
                books
            WHERE
                publication_date >= '2000-01-01' 
            ORDER BY
                publication_date
            ''' 

year_2000 = pd.io.sql.read_sql(query_2000, con = engine)
print(f"Cantidad de libros publicados después del 1° de enero del 2000: {len(year_2000)}")
display(year_2000.head(5))

query_2020 = ''' 
            SELECT 
                title,
                publication_date
            FROM 
                books
            WHERE
                publication_date >= '2020-01-01' 
            ORDER BY
                publication_date
            ''' 

year_2020 = pd.io.sql.read_sql(query_2020, con = engine)
print(f"Cantidad de libros publicados después del 1° de enero del 2020: {len(year_2020)}")
display(year_2020)

Cantidad de libros publicados después del 1° de enero del 2000: 821


,title,publication_date
0,Shopgirl,2000-01-01
1,A Room of One's Own,2000-01-01
2,Angels Flight (Harry Bosch #6; Harry Bosch Un...,2000-01-05
3,The Light Fantastic (Discworld #2; Rincewind #2),2000-02-02
4,The Gods Themselves,2000-02-10


Cantidad de libros publicados después del 1° de enero del 2020: 1


,title,publication_date
0,A Quick Bite (Argeneau #1),2020-03-31


Podemos observar que desde el 2000 han sido publicados por este servicio más de 800 libros. Solo uno de ellos es del 2020 publicado a comienzos de la pandemia.

In [ ]:
#Buscamos: cuantas reseñas hay por usuario
user_reviews = """
SELECT 
    username,
    COUNT(review_id) as reviews
FROM
    reviews
GROUP BY
    username
ORDER BY
    reviews DESC
LIMIT
10
"""

print('Cantidad de reseñas por usuario:')
r_p_u = pd.io.sql.read_sql(user_reviews, con = engine)
display(r_p_u)

Cantidad de reseñas por usuario:


,username,reviews
0,susan85,29
1,sfitzgerald,28
2,martinadam,27
3,eallen,26
4,lesliegibbs,26
5,tanya01,26
6,richard89,26
7,jennifermiller,25
8,mariokeller,25
9,amy97,25


Los usuarios en top 10, con mayor cantidad de reseñas, tienen entre 25 y 30 reseñas hechas.

In [ ]:
#por libro calculamos cantidad de reseñas y rating promedio

books_rr = """
        SELECT 
            books.title,
            COUNT(reviews.review_id) as reviews_amount,
            AVG(ratings.rating) as avg_rating
        FROM
            books
            INNER JOIN reviews ON books.book_id = reviews.book_id
            INNER JOIN ratings ON books.book_id = ratings.book_id
        GROUP BY
            books.book_id
        ORDER BY
            reviews_amount DESC
        LIMIT 10
"""

books_ra_re = pd.io.sql.read_sql(books_rr, con = engine)
print("Cantidad de reseñas por libro y rating promedio (si no aparece en la tabla es porque no posee reseña o rating)")
display(books_ra_re)


Cantidad de reseñas por libro y rating promedio (si no aparece en la tabla es porque no posee reseña o rating)


,title,reviews_amount,avg_rating
0,Twilight (Twilight #1),1120,3.662500
1,The Hobbit or There and Back Again,528,4.125000
2,The Catcher in the Rye,516,3.825581
3,Harry Potter and the Prisoner of Azkaban (Harr...,492,4.414634
4,Harry Potter and the Chamber of Secrets (Harry...,480,4.287500
5,Angels & Demons (Robert Langdon #1),420,3.678571
6,Harry Potter and the Order of the Phoenix (Har...,375,4.186667
7,The Lightning Thief (Percy Jackson and the Oly...,372,4.080645
8,The Fellowship of the Ring (The Lord of the Ri...,370,4.391892
9,Animal Farm,370,3.729730


Twilight (Crepúsculo) es, sin lugar a duda, el libro más popular en cuanto a cantidad de reseñas, no por eso el mejor puntuado. Ordenando los libros según cantidad de reseñas, ocupa el primer lugar (1120), sancando más del doble al libro que ocupa el segundo puesto: The Hobbit (528).

En un top 10, la mínima cantidad de reseñas por libro es 370. 

El rating promedio es entre 3.6 y 4.4.

In [ ]:
#Calculamos cantidad de libros publicados por editorial.
publisher = """ 
            SELECT 
                publishers.publisher,
                COUNT(books.num_pages) as books_amount
            FROM
                publishers
                INNER JOIN books ON publishers.publisher_id = books.publisher_id
            WHERE 
                books.num_pages > 50
            GROUP BY
                publishers.publisher
            ORDER BY
                books_amount DESC
            LIMIT 10
"""
publisher_books = pd.io.sql.read_sql(publisher, con = engine)
print('Cantidad de libros publicados por editorial:')
display(publisher_books)


Cantidad de libros publicados por editorial:


,publisher,books_amount
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,William Morrow Paperbacks,13


La editorial Penguin books, tiene más de 40 libros publicados (42), sacándole 11 de diferencia a Vintage que está segunda.

En el puesto 10, está William Morrow Paperbacks con 13 libros.

In [ ]:
#Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.
best_author = """
        SELECT 
            authors.author,
            COUNT(ratings.rating_id) as reviews_amount,
            AVG(ratings.rating) as avg_rating
        FROM
            authors
            INNER JOIN books ON authors.author_id = books.author_id
            INNER JOIN ratings ON books.book_id = ratings.book_id
        GROUP BY
            authors.author_id
        HAVING
            COUNT(ratings.rating_id) >= 50
        ORDER BY
            reviews_amount DESC
        LIMIT
        10
"""

author_bests = pd.io.sql.read_sql(best_author, con = engine)
display(author_bests)

,author,reviews_amount,avg_rating
0,J.K. Rowling/Mary GrandPré,312,4.288462
1,J.R.R. Tolkien,166,4.240964
2,Stephenie Meyer,160,3.662500
3,Dan Brown,143,3.741259
4,Nicholas Sparks,111,3.882883
5,Stephen King,106,4.009434
6,J.D. Salinger,98,3.846939
7,Rick Riordan,84,4.130952
8,Jodi Picoult,76,3.881579
9,George Orwell/Boris Grabnar/Peter Škerl,74,3.729730


J.K. Rownling es la autora con más de 300 reviews y un promedio de rating de 4.2, lo cual lo hace popular y aceptado por el público, es decir, con buen ranking.

Sin embargo, no todo autor con muchos reseñas tiene un buen ranking, por ejemplo Stephenie Meyer tiene 160 reseñas con un promedio de 3.6 y George Orwell/Boris Grabnar/Peter Škerl tienen 74 con un rating promedio de 3.7.

In [ ]:
#Calculamos promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.
users_reviews_50 = """  
            SELECT 
                AVG(count)
            FROM
                (SELECT
                ratings.username,
                COUNT(reviews.text)
            FROM
                ratings
                INNER JOIN reviews ON ratings.book_id = reviews.book_id
            GROUP BY
                ratings.username
            HAVING 
                COUNT(ratings.rating) > 50) AS count
    """

reviews_50 = pd.io.sql.read_sql(users_reviews_50, con = engine)
print('Promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros')
display(reviews_50)

Promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros


,avg
0,163.54375
